__Use case (WIP)__


This use case is will be showcasing how to use the Assisten API to chat with already compiled code which is provided as a nuget packaget. It will should work with any nuget that contains a dll and a xml file with a documentation in it.

__Prerequisits__

To run this sample either a [GPT-4 Turbo with Vision](https://learn.microsoft.com/en-us/azure/ai-services/openai/how-to/gpt-with-vision?tabs=rest%2Csystem-assigned%2Cresource) model or [GPT-4o](https://learn.microsoft.com/en-us/azure/ai-services/openai/concepts/models#gpt-4o-and-gpt-4-turbo) model is requiered.

In [ ]:
#r "nuget: Microsoft.SemanticKernel"

#r "nuget: Azure.ResourceManager.Storage"
#r "nuget: Azure.ResourceManager.KeyVault"

#!import config/Settings.cs
#!import plugins/ReflectionPlugin.cs
#!import helpers/AssemblyDocumentationHelper.cs

using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.ChatCompletion;
using Microsoft.SemanticKernel.Connectors.OpenAI;

In [ ]:
using Kernel = Microsoft.SemanticKernel.Kernel;

#pragma warning disable SKEXP0001, SKEXP0010

var settings = Settings.LoadFromFile("config/settings.json");
var builder = Kernel.CreateBuilder();

builder
    .AddAzureOpenAIChatCompletion(settings.model, settings.azureEndpoint, settings.apiKey)
    .Plugins.AddFromType<ReflectionPlugin>();

var kernel = builder.Build();
var chatGPT = kernel.GetRequiredService<IChatCompletionService>();

In [ ]:
var systemMessage = 
"""
You are a chat helping a c# developer getting inforametion about types.
""";

var chat = new ChatHistory(systemMessage);

__Streaming__

To use plugins we have to use a different GetMessage Method. 

Instead of using [GetChatMessageContentAsync](https://learn.microsoft.com/en-us/dotnet/api/microsoft.semantickernel.chatcompletion.ichatcompletionservice.getchatmessagecontentsasync?view=semantic-kernel-dotnet) we have to use [GetStreamingChatMessageContentsAsync](https://learn.microsoft.com/en-us/dotnet/api/microsoft.semantickernel.chatcompletion.ichatcompletionservice.getstreamingchatmessagecontentsasync?view=semantic-kernel-dotnet). If we would not change to the streaming method, semantic kernel is not able to call functions.

In [ ]:
var promptSettings = new OpenAIPromptExecutionSettings()
    {
        MaxTokens = 4000,
        ToolCallBehavior = ToolCallBehavior.AutoInvokeKernelFunctions,
    };

Func<string, Task> Chat = async (string input) => {
    chat.AddUserMessage(input);

     StringBuilder sb = new();
    await foreach (var message in chatGPT.GetStreamingChatMessageContentsAsync(chat, promptSettings, kernel))
    {
        sb.Append(message.Content);
    }
    var assistantReply = sb.ToString();
    chat.AddAssistantMessage(assistantReply);

    Console.WriteLine($"\nBot:");
    Console.WriteLine($"{assistantReply}\n");
};

In [ ]:
await Chat("Can you give me the path to the latest version of the Azure.ResourceManager.Storage assembly?");

In [ ]:
await Chat("Can you tell me something about the 'StorageAccountResource' in that assembly?");

In [ ]:
await Chat("Can you list me all the members of the 'StorageAccountResource' ?");

In [ ]:
await Chat("Can you tell me more aboute the 'CreateResourceIdentifier' method ?");